# Compute $\psi$ with APS POLAR geometry

Needs a custom build of `libhkl` with the `"APS POLAR"` geometry.

In [1]:
from hkl import Lattice
from hkl.calc import CalcRecip
from hkl.diffract import Diffractometer
from hkl.util import libhkl
from ophyd import Component as Cpt
from ophyd import PseudoSingle
from ophyd import SoftPositioner

print(f"{libhkl.VERSION=}")
print(f"{'APS POLAR' in libhkl.factories()=}")

libhkl.VERSION='5.0.0.3511'
'APS POLAR' in libhkl.factories()=True


In [2]:
class CalcApsPolar(CalcRecip):
    """Geometry: E6C"""

    def __init__(self, **kwargs):
        super().__init__("APS POLAR", **kwargs)

class SimulatedApsPolar(Diffractometer):
    """SimulatedApsPolar: APS POLAR 6-circle diffractometer, hkl engine"""

    calc_class = CalcApsPolar

    h = Cpt(PseudoSingle, "", kind="hinted")
    k = Cpt(PseudoSingle, "", kind="hinted")
    l = Cpt(PseudoSingle, "", kind="hinted")

    tau = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    mu = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    chi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    phi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    gamma = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    delta = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")

    def __init__(self, prefix, **kwargs):
        super().__init__(prefix, engine="hkl", **kwargs)


In [3]:
polar = SimulatedApsPolar("", name="polar")
polar.wh()

===================== ================================= =========
term                  value                             axis_type
===================== ================================= =========
diffractometer        polar                                      
sample name           main                                       
energy (keV)          8.05092                                    
wavelength (angstrom) 1.54000                                    
calc engine           hkl                                        
mode                  4-circles constant phi horizontal          
h                     0.0                               pseudo   
k                     0.0                               pseudo   
l                     0.0                               pseudo   
tau                   0                                 real     
mu                    0                                 real     
chi                   0                                 real     
phi       

Check the `"psi constant vertical"` mode is available.

In [4]:
"psi constant vertical" in polar.engine.modes

True

## Define a sample

In [5]:
a0 = 2 * 3.141592653589793
crystalline_vibranium = Lattice(a=a0, b=a0, c=a0, alpha=90, beta=90, gamma=90)
sample = polar.calc.new_sample("vibranium", lattice=crystalline_vibranium)
sample.compute_UB(
    sample.add_reflection(4, 0, 0, [0, 29.35, 0, 50, 0, 58.71]),
    sample.add_reflection(0, 4, 0, [0, 29.35, 0, -40, 0, 58.71]),
)

array([[-0.09059697, -0.63502298,  0.76716231],
       [ 0.48177505, -0.70213807, -0.52430423],
       [ 0.8715991 ,  0.32209928,  0.36954981]])

## Move to (111)

In [6]:
polar.move(1, 1, 1)
polar.engine.mode = "psi constant vertical"
print(f"{polar.engine.parameters=}")

polar.engine.parameters=['h2', 'k2', 'l2', 'psi']


Set azimuthal reflection $(110)$ and $\psi=12$.

In [7]:
polar.engine._engine.parameters_values_set([1, 1, 0, 12], 1)
print(f"{polar.engine._engine.parameters_values_get(1)=}")

polar.engine._engine.parameters_values_get(1)=[1.0, 1.0, 0.0, 12.0]


Compute the real-axis motor values with the $(111)$ reflection oriented and $\psi$ rotation around the azimuthal reflection.

In [8]:
p_111 = polar.forward(1, 1, 1)
print(f"{p_111=}")

p_111=PosCalcApsPolar(tau=-32.90008532040592, mu=-13.629264133644595, chi=63.55364766315855, phi=-108.99636837096102, gamma=-24.509844491665444, delta=49.32236362782608)


Move all reals to the $(111)$ reflection.

In [9]:
for axis in p_111._fields:
    getattr(polar, axis).move(getattr(p_111, axis))
print(f"{polar.position=}")
print(f"{polar.real_position=}")
print(f"{polar.engine._engine.parameters_values_get(1)=}")

polar.position=SimulatedApsPolarPseudoPos(h=1.0000000014935722, k=0.9999999989977246, l=0.9999999962820191)
polar.real_position=SimulatedApsPolarRealPos(tau=-32.90008532040592, mu=-13.629264133644595, chi=63.55364766315855, phi=-108.99636837096102, gamma=-24.509844491665444, delta=49.32236362782608)
polar.engine._engine.parameters_values_get(1)=[1.0, 1.0, 0.0, 12.0]


## Calculate $\psi$

Check the `"psi"` engine is available.

In [10]:
engines = [engine.name_get() for engine in polar.engine._engine_list.engines_get()]
print(f"engines=")
"psi" in engines

engines=


True

In [11]:
class SimulatedApsPolarPsi(Diffractometer):
    """SimulatedApsPolar: APS POLAR 6-circle diffractometer, psi engine"""

    calc_class = CalcApsPolar

    psi = Cpt(PseudoSingle, "", kind="hinted")

    tau = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    mu = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    chi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    phi = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    gamma = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")
    delta = Cpt(SoftPositioner, limits=(-180, 180), init_pos=0, kind="normal")

    def __init__(self, prefix, **kwargs):
        super().__init__(prefix, engine="psi", **kwargs)


In [12]:
polar_psi = SimulatedApsPolarPsi("", name="polar_psi")
print(f"{polar_psi.engine.mode=}")
print(f"{polar_psi.engine.parameters=}")

polar_psi.engine.mode='psi_vertical'
polar_psi.engine.parameters=['h2', 'k2', 'l2']


In [13]:
polar_psi.calc.new_sample(polar.calc.sample.name, lattice=polar.calc.sample.lattice)
polar_psi.calc.sample.UB = polar.UB.get()
polar_psi.engine._engine.parameters_values_set([1, 1, 0], 1)
for axis in p_111._fields:  # move all reals to the (111) reflection
    getattr(polar_psi, axis).move(getattr(p_111, axis))
print(f"{polar_psi.position=}")
print(f"{polar_psi.real_position=}")

polar_psi.position=SimulatedApsPolarPsiPseudoPos(psi=12.000080866657878)
polar_psi.real_position=SimulatedApsPolarPsiRealPos(tau=-32.90008532040592, mu=-13.629264133644595, chi=63.55364766315855, phi=-108.99636837096102, gamma=-24.509844491665444, delta=49.32236362782608)
